In [ ]:
import cloud_data.database_connector

list1= ["Select asdasl"]
list2= "Select dasahdka"
list3= ["asdasda","asdasda"]


print(type(list1))
print(type(list2))
print(type(list3))

In [ ]:
import database_connector as dc

In [ ]:
result = dc.query("SELECT count(column_name) as column_count FROM information_schema.columns WHERE table_name = 'test_table' and column_name != 'id'")[0][0]
print(f"--- {result}")

In [ ]:
result = dc.query("SELECT column_name as column_count FROM information_schema.columns WHERE table_name = 'test_table' and column_name != 'id'")
print(f">\n {result}")

list = tuple([column[0] for column in result])
print(list)

print(f"INSERT INTO table {list}")

In [ ]:
return_value = [('name',), ('bezeichnung',)]
count = len(return_value)
print(count)

new_list = []
for i in return_value:
    new_list.append(i[0])

print(new_list)

new_list2 = [column[0] for column in return_value]
print(new_list2)

filtered = len(return_value)
print(f"({('%s,'*(filtered-1) + '%s')})")


In [ ]:
values1 = None
values = ()

def ubs():
    if not values or len(values) < 1:
        return ("Please provide a 'value' when using this function")

ubs()

In [ ]:
def generate_create_statement(table: str, values: tuple[tuple[str, str], ...]) -> str:
    columns = []
    for column_name, column_type in values:
        columns.append(f"{column_name} {column_type}")

    columns_sql = ",\n\t".join(columns)
    statement = f"CREATE TABLE {table} (\n\t{columns_sql}\n);"
    return statement


table = "ernst"
values = (
    ("id", "SERIAL PRIMARY KEY"),
    ("name", "varchar(255)"),
    ("type", "varchar(100)")
)

print(generate_create_statement(table,values))

## Database tests
------------------------

In [ ]:
import database_connector as dc

#### Query

In [ ]:
statement = "SELECT * FROM test"
print(dc.query(statement))

#### Insert

In [ ]:
#Test mit .execute()

table = "test"
# Table test hat die Columns "name", "number" ist alphabetisch
values = ('Ivan',22)

print(dc.insert(table=table,values=values))

In [ ]:
#Test mit .executemany()

table = "test"
# Table test hat die Columns "name", "number" ist alphabetisch
slist = [('Ivan',30),('Thomas',34),('Stefan',40)]

print(dc.insert(table=table,slist=slist))

## Google Cloud Storage (Bucket)
--------------

### Initialize cloud_connector

In [ ]:
import cloud_connector as cc

### Upload

In [ ]:
# Full Filepath with filename
cc.upload_to_bucket("../airflow/test25.txt")

### Download

In [ ]:
# Filename in Bucket and Filelocation
cc.download_from_bucket("test25.txt","test25.txt")